In [10]:
%load_ext autoreload
%autoreload 2

%pip install -r requirements.txt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Note: you may need to restart the kernel to use updated packages.


In [11]:
import sys
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision
import torchvision.models as models
from torchvision import transforms
from datasets import load_dataset, concatenate_datasets

In [12]:
print(f"PyTorch version: {torch.__version__}")

# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")

# Check for CUDA support
print(f"Is CUDA available? {torch.cuda.is_available()}")

# Set the device
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Using device: {device}")


PyTorch version: 2.1.0
Is MPS (Metal Performance Shader) built? True
Is MPS available? True
Is CUDA available? False
Using device: mps


In [13]:
# pick which model to load
model_name = "vgg_cifar100" # either "resnet" or "vgg_cifar10" or "vgg_cifar100"
num_classes = 100 if model_name == "vgg_cifar100" else 10
model_path = os.path.join("models", model_name)

In [14]:
from DataLoader import CustomDataset
import numpy as np

if model_name == "resnet":
    # use the imagenette dataset
    hf_dataset = load_dataset("frgfm/imagenette", '320px')
    hf_dataset = concatenate_datasets(hf_dataset.values())
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

elif model_name == "vgg_cifar10":
    # use the cifar10 dataset
    hf_dataset = load_dataset("cifar10")
    hf_dataset = concatenate_datasets(hf_dataset.values())
    
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.507, 0.4865, 0.4409],
                             std=[0.2673, 0.2564, 0.2761])
    ])
elif model_name == "vgg_cifar100":
    # use the cifar100 dataset
    hf_dataset = load_dataset("cifar100")
    hf_dataset = concatenate_datasets(hf_dataset.values())
    
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
    ])
    
torch_dataset = CustomDataset(hf_dataset, transform=transform)

batch_size = 32 if model_name == "resnet" else 64

test_size = 0.2
test_volume = int(test_size * len(torch_dataset))
train_volume = len(torch_dataset) - test_volume

train_dataset, test_dataset = random_split(torch_dataset, [train_volume, test_volume])
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=False, 
    num_workers=4
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4
)

In [15]:
import ModelLoader

loader = ModelLoader.ModelLoader(model_name, device, alpha=0.8, dataloader=train_dataloader)

# preview the model architecture
model = loader.load_model(num_outputs=num_classes, trained_classifiers=True)
model

Loading EarlyExit VGG11 model architecture...
Adding exits...


Using cache found in /Users/dylanmace/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


Setting model weights...


EarlyExitModel(
  (model): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU(inplace=True)
      (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (8): OptionalExitModule(
        (module): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (exit_gate): Linear(in_features=8192, out_features=1, bias=True)
        (classifier): Linear(in_features=8192, out_features=100, bias=True)
      )
      (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

## Early Exit Model Training

If you want tensorboard support, you need to run the following commands on **macOS**:

This clears your logs:
```
rm -rf runs/*
```

This shows your public IP address:
```
dig -4 TXT +short o-o.myaddr.l.google.com @ns1.google.com
```

This starts tensorboard at this address:
```
tensorboard --host 0.0.0.0 --logdir={model_path}/runs
```
```



If you want tensorboard support, you need to run the following commands on **Windows**:

This clears your logs:
```
rmdir /s /q runs
```

This shows your IP address:
```
powershell -command "$ipAddress = (Invoke-WebRequest -Uri 'http://ipinfo.io/ip').Content.Trim(); Write-Host 'Your IP address is: ' $ipAddress"  
```

This starts tensorboard at this address:
```
tensorboard --host 0.0.0.0 --logdir={model_path}\runs
```

In [16]:
from EarlyExitTrainer import ModelTrainer

trainer = ModelTrainer(model, device, model_dir=model_path)

In [17]:
# train the exits
alpha_range = [i / 100 for i in range(0, 101, 10)] #[0, 0.25, 0.5, 0.55, 0.6, 0.65, 0.7, 0.725, 0.75, 0.775] + [i / 100 for i in range(80, 101, 2)]

accuracies = {}
times = {}
avg_exit_idx = {}


epoch_count = 3 if model_name == "resnet" else 3
lr = 0.00000001 if model_name == "resnet" else 0.000001

for alpha in alpha_range:
    print(f"Training with alpha={alpha}")
    trainer.model = loader.load_model(num_outputs=num_classes, trained_classifiers=True) # reset the model each time
    trainer.set_alpha(alpha)
    final_accuracy, final_time, final_avg_exit_idx = trainer.train_exit_layers(train_dataloader, lr, epoch_count=epoch_count, validation_loader=test_dataloader)
    
    # write statistics
    accuracies[alpha] = final_accuracy
    times[alpha] = final_time
    avg_exit_idx[alpha] = final_avg_exit_idx


Training with alpha=0.0
Loading EarlyExit VGG11 model architecture...
Adding exits...


Using cache found in /Users/dylanmace/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


Setting model weights...


Epoch 0:   3%|█▎                                        | 24/750 [00:59<15:25,  1.28s/it, Loss=3.77]

In [ ]:
    
# generate dataframe containing statistics
import pandas as pd

df = pd.DataFrame({
    "accuracy": accuracies,
    "time": times,
    "avg_exit_idx": avg_exit_idx
})

df

In [ ]:
# save dataframe

df.to_csv(os.path.join(model_path, "alpha_tuning.csv"), index=False)